In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import random

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
  test_labels_without_reformat = test_labels.copy()

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def confusion_matrix(ori_labels, pred_labels):
    confision_matrix = np.zeros((10, 10))
    for ori_lab, pred_lab in zip(ori_labels, pred_labels):
        confision_matrix[ori_lab, pred_lab] += 1
    return confision_matrix

def recall(confusion_mat,lab):
    #print("True Positives : {} \nTrue Positives + False Negetives : {}".format(confusion_mat[lab,lab],np.sum(confusion_mat[:,lab])))
    rec = confusion_mat[lab,lab]/np.sum(confusion_mat[:,lab])
    #print("Recall for class {} : {}".format(lab,rec))
    return rec

def precision(confusion_mat,lab):
    #print("True Positives : {} \nTrue Positives + False Positives : {}".format(confusion_mat[lab,lab],np.sum(confusion_mat[lab,:])))
    prec = confusion_mat[lab,lab]/np.sum(confusion_mat[lab,:])
    #print("Precision for class {} : {}".format(lab,prec))
    return prec

def f1_score(confusion_mat,lab):
    return 2*(precision(confusion_mat,lab)*recall(confusion_mat,lab)) / (precision(confusion_mat,lab) + recall(confusion_mat,lab))

def plot_recall_n_precision(confusion_mat):
    """
    Check the number of images under each class or label
    and plot a histogram to see if the dataset is balanced
    """
    print("Confusion Matrix : \n{}".format(confusion_mat))
    recall_list = [recall(confusion_mat,x) for x in range(10)]
    precision_list = [precision(confusion_mat,x) for x in range(10)]
    f1score_list = [f1_score(confusion_mat,x) for x in range(10)]
    
    plt.figure()
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
    #print(axes) 
    axes[0].bar(range(10),precision_list)
    labels = string.uppercase[:10]
    axes[0].set_xticks([x + 0.5 for x in range(10)])
    axes[0].set_xticklabels(labels)
    axes[0].set_ylabel('Precision')
    axes[0].set_xlabel('Image Classes')
    axes[0].set_title('Precision Plot')
    
    axes[1].bar(range(10),recall_list)
    axes[1].set_xticks([x + 0.5 for x in range(10)])
    axes[1].set_xticklabels(labels)
    axes[1].set_ylabel('Recall')
    axes[1].set_xlabel('Image Classes')
    axes[1].set_title('Recall Plot')
    
    axes[2].bar(range(10),f1score_list)
    axes[2].set_xticks([x + 0.5 for x in range(10)])
    axes[2].set_xticklabels(labels)
    axes[2].set_ylabel('F1 Score')
    axes[2].set_xlabel('Image Classes')
    axes[2].set_title('F1 Score Plot')
    
    fig.tight_layout()
    plt.show()

In [121]:
batch_size = 128
image_size = 28
num_labels = 10
hidden_layer_1_count = 1024
hidden_layer_2_count = 512
hidden_layer_3_count = 256
hidden_layer_4_count = 128

batch_size = 256
logs_name = "mnist_logs"

import shutil
try:
    shutil.rmtree(logs_name)
except: 
    print("Unable to delete the dir {}".format(logs_name))

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size),name="training_data")
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels),name="training_labels")
    tf_valid_dataset = tf.constant(valid_dataset,name="validation_data")
    tf_test_dataset = tf.constant(test_dataset,name="test_data")
    pred_dataset = tf.placeholder(tf.float32,shape=(10000, image_size * image_size),name="pred_data")
    pred_label = tf.placeholder(tf.float32, shape=(10000, num_labels),name="pred_labels")
    lamda_i = tf.constant(0.01)
        
    ##################################
    # Input Layer
    ##################################
    
    with tf.name_scope('input_layer'):
        with tf.name_scope('weight'):
            W_i = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]),name="weight")
        with tf.name_scope('Bias'):
            B_i = tf.Variable(tf.zeros([num_labels]))
        with tf.name_scope('logits'):    
            input_logits = tf.matmul(tf_train_dataset, W_i) + B_i
            
    ##################################
    # Hidden Layer 1
    ##################################
    
    with tf.name_scope('hidden_layer_1'):
        with tf.name_scope('weight'):
            W_h_1 = tf.Variable(tf.truncated_normal([num_labels,hidden_layer_1_count]),name="weight")
        with tf.name_scope('Bias'):
            B_h_1 = tf.Variable(tf.zeros([hidden_layer_1_count]))
        with tf.name_scope('logits'):    
            hidden_logit_1 = tf.matmul(tf.nn.relu6(input_logits),W_h_1) + B_h_1
            
            
    
    ##################################
    # Output Layer 
    ##################################
    
    with tf.name_scope('output_layer'):
        with tf.name_scope('weight'):
            W_o = tf.Variable(tf.truncated_normal([hidden_layer_1_count,num_labels]),name="weight")
        with tf.name_scope('Bias'):
            B_o = tf.Variable(tf.zeros([num_labels]))
        with tf.name_scope('logits'):    
            output_logit = tf.matmul(tf.nn.relu6(hidden_logit_1),W_o) + B_o    
    
            
    ##################################
    # SGD Optimizer
    ##################################         
    with tf.name_scope('sgd_optimizer'):
        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_logit , tf_train_labels))
        with tf.name_scope('sgd'):
            optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        
    tf.histogram_summary("Loss",loss)
        
    with tf.name_scope('prediction_layer'):
        train_prediction = tf.nn.softmax(output_logit)
        for_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu6(tf.matmul(tf.nn.relu6(tf.matmul(pred_dataset, W_i)+ B_i),W_h_1) + B_h_1),W_o) + B_o)
        for_train_accuracy = tf.placeholder(tf.float32,name="train_accuracy")
        for_valid_accuracy = tf.placeholder(tf.float32,name="valid_accuracy")
    tf.histogram_summary("Train Accuracy",for_train_accuracy)
    tf.histogram_summary("Validation Accuracy",for_valid_accuracy)   
    #im_t = tf.placeholder(tf.float32, shape=[None,28,28,1], name='img_tensor')
    #tf.image_summary("img", im_t)
    merged = tf.merge_all_summaries()
    
            

In [122]:
num_steps = 1000

with tf.Session(graph=graph) as session:
  writer = tf.train.SummaryWriter(logs_name, session.graph_def)
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    #print(predictions.type)
    #break
    
   
    if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            #print (for_prediction.eval(feed_dict={pred_dataset : valid_dataset}))
            #valid_acc = accuracy(for_prediction.eval(feed_dict={tf_train_dataset : tf_valid_dataset}), valid_labels)
            #print(tf.expand_dims(valid_dataset[1].reshape((28, 28,1)),0).eval())
            #print(valid_dataset[1].reshape((28, 28,1)))
            disp_dict = {tf_train_dataset : batch_data,
                         tf_train_labels : batch_labels,
                         for_train_accuracy: accuracy(predictions, batch_labels),
                         for_valid_accuracy: accuracy(for_prediction.eval(feed_dict={pred_dataset : valid_dataset}), valid_labels),
                        }
                         #im_t : tf.expand_dims(valid_dataset[1].reshape((28, 28,1)),0).eval()
                        #}
            summary_str = merged.eval(feed_dict = disp_dict)
            writer.add_summary(summary_str, step)
            #train_accuracy = accuracy(predictions, batch_labels)
            #print(accuracy(predictions, batch_labels))
            #writer.add_summary(bla.eval(feed_dict={train_accuracy: accuracy(predictions, batch_labels)}), step)
            
            #print("chiraku dobbutundi %d : %f ")% (step,train_accuracy.eval(feed_dict={pred_dataset : valid_dataset, pred_label : valid_labels}))
            #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            #print("Validation accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={pred_dataset : valid_dataset, pred_label : valid_labels}),valid_labels))
    #except:
    #    print("Training interrepted : Stupid bug")
  pred_list = for_prediction.eval(feed_dict={pred_dataset : test_dataset})
  print("Test accuracy: %.1f%%" % accuracy(pred_list, test_labels.copy()))

Initialized
Minibatch loss at step 0: 101.975395
Minibatch loss at step 500: 3.647011
Test accuracy: 22.4%
